# [Using EasyOCR with pretrained YOLO for License Plate Recognition](https://youtu.be/9KnARRqbkzY?si=lwrQjDCc9q9ERes-&t=1560)

---

This notebook demonstrates how to use EasyOCR in combination with a pretrained YOLO model to perform license plate recognition. The YOLO model is used to detect license plates in images, and EasyOCR is then used to read the text from the detected plates.


Before running a notebook for the first time, install the required dependencies with:

```bash
	make requirements-nb
```


In [ ]:
from civicpulseML.config import BASE_DIR, CUDA_AVAILABLE, MODELS_DIR, RAW_DIR

## Importing finetuned YOLO model and EasyOCR reader

---


In [2]:
from ultralytics import YOLO
import json

### Reading metadata file


In [3]:
with open(MODELS_DIR / "modeldata.json") as f:
    model_config = json.load(f)

### Getting path of best model


In [4]:
best_path = model_config["yolo"]["weights_path"]

### Loading YOLO model


In [6]:
model = YOLO(BASE_DIR / best_path)

### Initialising EasyOCR Reader


In [9]:
import easyocr

reader = easyocr.Reader(
    lang_list=['en'],
    model_storage_directory=MODELS_DIR / "easyocr",
    gpu=CUDA_AVAILABLE
)

## Defining correction patterns for license plates

---


### Correction using Regex


License plates often follow specific formats depending on the region or country. To improve the accuracy of OCR results, we can define correction patterns using regular expressions that help in validating and correcting the recognized text.

For instance, in India, a common license plate format is "XX00XX0000", where "X" represents letters and "0" represents digits.

The following regex pattern captures standard vehicle numbers:-


In [10]:
plate_re = r"^[A-Z]{2}[0-9]{2}[A-Z]{3}$"

In [11]:
import re

plate_pattern = re.compile(plate_re)

### Correction by Mapping to closest number/letter


In [12]:
def correct_plate_format(ocr_text):
    """
    Corrects the OCR text to match the license plate format using regex.
    If the text matches the pattern, it is returned as is.
    If not, attempts to correct common OCR misinterpretations.
    Args:
        ocr_text (str): The text recognized by OCR.
    Returns:
        str: Corrected license plate text or empty string if correction fails.
    """

    mapping_num_to_alpha = {0: 'O', 1: 'I', 5: 'S', 8: 'B', 2: 'Z'}
    mapping_alpha_to_num = {'O': 0, 'I': 1, 'S': 5, 'B': 8, 'Z': 2}

    ocr_text = ocr_text.upper().replace(" ", "")
    if len(ocr_text) != 7:
        return ocr_text  # discard if wrong length

    corrected = []
    for i, ch in enumerate(ocr_text):
        if i < 2 or i >= 4:  # alphabet positions
            if ch.isdigit() and int(ch) in mapping_num_to_alpha:
                corrected.append(mapping_num_to_alpha[int(ch)])
            elif ch.isalpha():
                corrected.append(ch)
            else:
                return ""  # invalid char
        else:  # numeric positions
            if ch.isalpha() and ch in mapping_alpha_to_num:
                corrected.append(str(mapping_alpha_to_num[ch]))
            elif ch.isdigit():
                corrected.append(ch)
            else:
                return ""  # invalid char
    
    return "".join(corrected)

## Preprocessing License Plate

---


In [13]:
import cv2
import numpy as np

In [ ]:
def recognize_plate(plate_crop):
    """Recognizes and corrects the license plate text from the cropped image."""
    
    if plate_crop.size == 0:
        return ""
    
    # Preprocess for OCR
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    plate_resized = cv2.resize(thresh, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    try:
        ocr_result = reader.readtext(plate_resized, detail=0, allowlist="ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
        if len(ocr_result) > 0:
            candidate = correct_plate_format(ocr_result[0])
            if candidate and plate_pattern.match(candidate):
                return candidate
    except Exception as e:
        print(f"OCR Error: {e}")

    return ""